## Download required Libraries

In [1]:
#@markdown # **1) Download required Libraries**
# # Clone Real-ESRGAN and enter the Real-ESRGAN
from google.colab import drive, files
import os, shutil, subprocess
drive_mounted = False
temp_folder = 'tmp_frames'
result_folder = 'results'
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
%cd ..

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 755, done.
remote: Total 755 (delta 0), reused 0 (delta 0), pack-reused 755
Receiving objects: 100% (755/755), 5.37 MiB | 3.24 MiB/s, done.
Resolving deltas: 100% (410/410), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.9 MB/s eta 0:00:00
  Created

## Upload Video using the DIALOG box!

In [3]:
if drive_mounted is not True:

  upload_folder = 'upload'

  if os.path.isdir(upload_folder):
      shutil.rmtree(upload_folder)

  os.mkdir(upload_folder)

  # upload images
  uploaded = files.upload()
  file_name = next(iter(uploaded))
  input_path = f'/content/{upload_folder}/{file_name}'
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)
else:
  print("drive mounted nothing to do here move along")

Saving test1.mp4 to test1.mp4
move test1.mp4 to upload/test1.mp4


 ## Extract Frames from Video

In [4]:
if os.path.isdir(temp_folder):
  shutil.rmtree(temp_folder)

os.mkdir(temp_folder)
print(f'Extracting Frames to: {temp_folder}')
cmd = [
       'ffmpeg',
       '-i',
       input_path,
       '-qscale:v',
       '1',
       '-qmin',
       '1',
       '-qmax',
       '1',
       '-vsync',
       '0',
       f'{temp_folder}/frame_%08d.png'
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    frame_count = len(os.listdir(f'/content/{temp_folder}'))
    print(f"Done, Extracted {frame_count} Frames")

Extracting Frames to: tmp_frames
Done, Extracted 823 Frames


## Run ERSGAN on Extracted Frames!

In [5]:
%cd /content/Real-ESRGAN
frame_count = len(os.listdir(f'/content/{temp_folder}'))
print(f"Enhancing {frame_count} Frames with ESRGAN...")
cmd = [
    'python',
    'inference_realesrgan.py',
    '-n',
    'RealESRGAN_x4plus',
    '-i',
    f'../{temp_folder}',
    '--outscale',
    '4',
    '--face_enhance'
]
#process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# stdout, stderr = process.communicate()
# if process.returncode != 0:
#     print(stderr)
#     raise RuntimeError(stderr)
# else:
#     print("Done Extracting Frames")
!{' '.join(cmd)}
print("Done Enhancing Frames")
%cd ..

/content/Real-ESRGAN
Enhancing 823 Frames with ESRGAN...
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 362MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated s

## Recreate Video From Enhanced Frames!

In [6]:
import subprocess
frame_count = len(os.listdir(f'/content/{temp_folder}'))
if os.path.isdir(result_folder):
  shutil.rmtree(result_folder)
os.mkdir(result_folder)

fps = 15 #@param {type: "number"}
print(f"Recompiling {frame_count} Frames into Video...")
cmd = [
       'ffmpeg',
       '-i',
       f'/content/Real-ESRGAN/results/frame_%08d_out.png',
       '-c:a',
       'copy',
       '-c:v',
       'libx264',
       '-r',
       str(fps),
       '-pix_fmt',
       'yuv420p',
       f'{result_folder}/enhanced_{file_name}'
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Done Recreating Video")

Recompiling 823 Frames into Video...
Done Recreating Video


# To extract the audio from the original video

In [10]:
import subprocess

input_video_path = '/content/upload/test1.mp4'  # Replace with the actual path to your video file
output_audio_path = '/content/upload/output_audio.aac'  # Replace with the desired output audio file path

# Command to extract audio from video
cmd_extract_audio = [
    'ffmpeg',
    '-i', input_video_path,
    '-vn',  # Disable video recording
    '-acodec', 'copy',  # Copy audio codec
    output_audio_path
]

subprocess.run(cmd_extract_audio)

CompletedProcess(args=['ffmpeg', '-i', '/content/upload/test1.mp4', '-vn', '-acodec', 'copy', '/content/upload/output_audio.aac'], returncode=0)

## To map the audio.acc file to the enhanced video file

In [12]:
import subprocess

frame_count = len(os.listdir(f'/content/{temp_folder}'))
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(result_folder)

fps = 15 #@param {type: "number"}
print(f"Recompiling {frame_count} Frames into Video...")

# Specify the path to the original audio file
audio_path = '/content/upload/output_audio.aac'

cmd = [
    'ffmpeg',
    '-i', f'/content/Real-ESRGAN/results/frame_%08d_out.png',
    '-i', audio_path,  # Include the extracted audio file
    '-c:a', 'aac',  # Set the audio codec (you can change it based on your needs)
    '-c:v', 'libx264',
    '-r', str(fps),
    '-pix_fmt', 'yuv420p',
    '-map', '0:v',  # Map video from the first input
    '-map', '1:a',  # Map audio from the second input
    '-shortest',  # End encoding when the shortest input stream ends
    f'{result_folder}/enhanced_{file_name}'
]

process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Done Recreating Video with Extracted Audio")


Recompiling 823 Frames into Video...
Done Recreating Video with Extracted Audio
